## Langchain Pandas Agent

Langchain agent with Python tool for dataframe and conversation memory.

In [8]:
from langchain.agents import create_pandas_dataframe_agent
from langchain.chat_models import ChatOpenAI
from langchain.agents.agent_types import AgentType

In [9]:
from langchain.prompts import MessagesPlaceholder
from langchain.memory import ConversationBufferMemory

agent_kwargs = {
    "extra_prompt_messages": [MessagesPlaceholder(variable_name="memory")],
}
memory = ConversationBufferMemory(memory_key="memory", return_messages=True)

In [10]:
from langchain.llms import OpenAI
import pandas as pd

df = pd.read_csv("Sport car price.csv")

In [11]:
agent = create_pandas_dataframe_agent(
    ChatOpenAI(temperature=0, model="gpt-3.5-turbo-0613"),
    df,
    verbose=True,
    agent_type=AgentType.OPENAI_FUNCTIONS,
    agent_executor_kwargs={"agent_kwargs": agent_kwargs, "memory": memory},
)

In [12]:
agent.run("What is the format of the dollar amount in the price column? Is there a comma for thousands?")



> Entering new AgentExecutor chain...



Invoking: `python_repl_ast` with `df['Price (in USD)'].dtype`


object
Invoking: `python_repl_ast` with `df['Price (in USD)'].head().tolist()`


['101,200', '274,390', '333,750', '142,700', '298,000']The format of the dollar amount in the price column is a string. It includes a comma for thousands.

> Finished chain.


'The format of the dollar amount in the price column is a string. It includes a comma for thousands.'

In [13]:
agent.run("What is the average price of Ferrai?")



> Entering new AgentExecutor chain...



Invoking: `python_repl_ast` with `df[df['Car Make'] == 'Ferrari']['Price (in USD)'].mean()`


TypeError: Could not convert 333,750625,000222,000625,000222,000274,280625,000222,000219,750276,550222,620507,000276,550274,280625,000363,730218,750625,000625,000276,550218,750274,280218,750215,000280,000625,000274,280625,000245,000222,630625,000625,000340,000625,000222,630625,000625,000625,000250,000245,000274,280625,000235,000625,000625,000625,000222,620625,000275,000274,390625,000274,280625,000333,750625,000 to numeric
Invoking: `python_repl_ast` with `df[df['Car Make'] == 'Ferrari']['Price (in USD)'].str.replace(',', '').astype(float).mean()`
responded: {content}

410099.0909090909The average price of Ferrari is $410,099.09.

> Finished chain.


'The average price of Ferrari is $410,099.09.'

In [14]:
memory.buffer

[HumanMessage(content='What is the format of the dollar amount in the price column? Is there a comma for thousands?', additional_kwargs={}, example=False),
 AIMessage(content='The format of the dollar amount in the price column is a string. It includes a comma for thousands.', additional_kwargs={}, example=False),
 HumanMessage(content='What is the average price of Ferrai?', additional_kwargs={}, example=False),
 AIMessage(content='The average price of Ferrari is $410,099.09.', additional_kwargs={}, example=False)]

## Langchain Custom Agent

Customized langchain agent with Python tool and human tool.

In [19]:
from langchain.agents import load_tools
from langchain.agents import initialize_agent
from langchain.agents import AgentType
from langchain.chat_models import ChatOpenAI
from langchain.agents.agent_types import AgentType
from langchain.chat_models import ChatOpenAI
from langchain.prompts import MessagesPlaceholder
from langchain.memory import ConversationBufferMemory

In [20]:
llm = ChatOpenAI(temperature=0, model="gpt-3.5-turbo-0613")
tools = load_tools(["python_repl", "human"])
agent_kwargs = {
    "extra_prompt_messages": [MessagesPlaceholder(variable_name="memory")],
}
memory = ConversationBufferMemory(memory_key="memory", return_messages=True)
agent = initialize_agent(tools=tools, llm=llm, agent_type=AgentType.OPENAI_FUNCTIONS, verbose=True, agent_kwargs=agent_kwargs, memory=memory)

In [22]:
agent.run("Read the data from 'Sport car price.csv'. Then calculate the average price of Ferrai.")



> Entering new AgentExecutor chain...
I need to read the data from the CSV file and then calculate the average price of Ferraris. I can use the Python CSV module to read the data and then iterate over the rows to calculate the average price.
Action: Python_REPL
Action Input: import csv
Observation: 
Thought:I have imported the csv module. Now I need to open the file and read the data.
Action: Python_REPL
Action Input: with open('Sport car price.csv', 'r') as file:
    reader = csv.reader(file)
    next(reader) # skip the header row
    prices = []
    for row in reader:
        if row[0] == 'Ferrari':
            prices.append(float(row[1]))
Observation: ValueError("could not convert string to float: '488 GTB'")
Thought:There seems to be an issue with converting the price to a float. I need to investigate further.
Action: Python_REPL
Action Input: print(row[1])
Observation: 488 GTB

Thought:The issue is that the price value for the Ferrari '488 GTB' is not a valid float. I need to fi

'No Ferrari prices found in the dataset.'